<a href="https://colab.research.google.com/github/alienlifeform/exotic-colab/blob/main/saved_colabs/exo-refactor2_EXOTIC_See_it_in_Action.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **EXOTIC: See it in Action**

Following is a demonstration of how to run EXOTIC's code in your browser on a set of sample telescope images to generate a lightcurve.

Steps:

1. LOAD TELESCOPE IMAGES
 * For this tutorial, we'll use images taken from a previous transition of planet HAT-P-32 b
1. IDENTIFY TARGET AND COMPARISON STARS IN A TELESCOPE IMAGE
 * Find the target star (HAT-P-32) and several comparison stars that have stable brightness
 * Enter their coordinates to calibrate the baseline brightness
1. RUN EXOTIC TO GENERATE A LIGHTCURVE
 * Watch EXOTIC determine if the brightness of your target star changes, signifying a transit. A transit is when an exoplanet passes in front of its host star, momentarily dimming out the star's light. [(See an animation of a transit)](https://exoplanets.nasa.gov/alien-worlds/ways-to-find-a-planet/#/2)

INSTRUCTIONS

Click the Screen Shot 2022-09-08 at 10.19.58 AM.png  icons below to run the code for each step. Wait for it to finish running before moving to the next step, and watch for the output describing what is happening.

```
Important: During this tutorial, it is recommended that you do not click the blue "Show code" link. 
```

---

## Step 1: Load Telescope Images

🕔 Estimated time: 2 minutes

For this tutorial, we are using sample images from a known transit. When you run it yourself, you will upload your own images.

```
Important:
  • A popup will appear titled "Warning: This notebook was not authored by Google.". Read it, then click "Run Anyway".
  • Ignore any warning that may ask you to "RESTART RUNTIME"
```

In [ ]:
#@title <font size=3><img src="https://exoplanets.nasa.gov/system/exotic/leftdownarrow_tall.png" height=18 hspace=8>Load telescope images</font>

##############################################################
%%capture step_capture --no-display
#display('Starting application, please wait...')

from IPython.display import display, HTML, Javascript
display(HTML('<p class="bookend">START: Importing necessary software libraries</p>'))
display(HTML('<p class="hidden">Loading styles, please wait...</p>'))

#display('Loading styles...')
#display(HTML('<li class="step">Loading styles...</li>'))

# Install EXOTIC Colab interface code
!pip install git+https://github.com/alienlifeform/exotic-colab.git --upgrade

from exoticcolab.display import testImplementation, importCustomStyles, setupDisplay, displayStep, makeContainer, appendToContainer, expandableSection, showProgress,resize_colab_cell
#testImplementation()

# Set up custom Colab styles and interactions
importCustomStyles()
setupDisplay()
get_ipython().events.register('pre_run_cell', resize_colab_cell)

#display(HTML('<p class="bookend">START: Importing necessary software libraries</p>'))

display(HTML('<ul class="step_container"></ul>'))

displayStep('(1/5) Bokeh.io')
import bokeh.io

displayStep('(2/5) EXOTIC <span class="comment">(This will take up to a minute, please wait...)</span>')

#!pip install exotic --upgrade
!pip install git+https://github.com/alienlifeform/exotic-prototype.git --upgrade
display(HTML('<br /><p>If there is a "RESTART RUNTIME" warning button above, ignore it!</p>'))

# from exotic.api.plotting import plot_image

displayStep('(3/5) NASAExoplanetArchive, Astropy, Utils')
from exotic.exotic import NASAExoplanetArchive, get_wcs, find_target
from astropy.time import Time
from barycorrpy import utc_tdb
import numpy as np
from io import BytesIO
from astropy.io import fits
from scipy.ndimage import label
from bokeh.plotting import figure, output_file, show
from bokeh.palettes import Viridis256
from bokeh.models import ColorBar, LinearColorMapper, LogColorMapper, LogTicker
from bokeh.models import BoxZoomTool,WheelZoomTool,ResetTool,HoverTool,PanTool,FreehandDrawTool
from bokeh.io import output_notebook
from pprint import pprint
from IPython.display import Image
from ipywidgets import widgets, HBox
from skimage.transform import rescale, resize, downscale_local_mean
import copy
import os
import re
import json
import subprocess

displayStep('(4/5) Matlab, SciPy')
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

displayStep('(5/5) Google Utils')
from google.colab import drive, files

#expandableSection('<p>This is some expandable stuff</p>')
display(HTML('<p class="bookend">DONE: Importing necessary software libraries</p>'))


#########################################################

display(HTML('<p class="bookend">START: Loading telescope images</p>'))
display(HTML('<ul class="step_container"></ul>'))

display(HTML('<li class="step">Ensuring sample images are loaded...</li>'))

#
# Delete existing sample data by changing to `rebuild = "true"`
#
rebuild = "true"  
if rebuild == "true":
  if os.path.isdir("/content/EXOTIC/exotic-in-action"):
    %rm -rf /content/EXOTIC/exotic-in-action
    display(HTML('<p class="step">"Rebuild" flag is "true"... Removing old images at /content/EXOTIC/exotic-in-action</p>'))

#
# Download sample files if necessary
#
#sample_data_source = "https://github.com/rzellem/EXOTIC_sampledata.git" # goes into sample_data_target_folder
sample_data_source = "https://github.com/alienlifeform/exotic-sample-data.git" # goes into sample_data_target_parent
sample_data_target_parent = "/content/EXOTIC/exotic-in-action"
sample_data_target_folder = "/content/EXOTIC/exotic-in-action/sample-data"
sample_data_target_child = "/content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017"
sample_data_target_outputs = "/content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output"
if os.path.isdir(sample_data_target_child):
  display(HTML('<li class="step">Skipping... Sample images already loaded</li>'))

else:
  display(HTML('<li class="step">Downloading images from ' + sample_data_source + '</li>'))
  #git_rv = !git clone {sample_data_source} {sample_data_target_folder}
  git_rv = !git clone {sample_data_source} {sample_data_target_parent}
  git_co_rv = !cd {sample_data_target_parent} && !git checkout beta1
  display(HTML('<li class="step">Telescope images successfully loaded for HAT-P-32 b</li>'))

#
# Show files found
#
#!du -hd0 --exclude ".*" /content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017
numfiles_fits = !ls {sample_data_target_child} | grep -ci FITS
numfiles_json = !ls {sample_data_target_child} | grep -ci json

display(HTML('<p class="step">You have ' + str(numfiles_fits[0]) + ' telescope image (.FITS) files</p>'))
display(HTML('<p class="step">You have ' + str(numfiles_json[0]) + ' inits (.json) files</p>'))

display(HTML('<p class="bookend">DONE: Loading telescope images. <b>You may move on to the next step.</b></p>'))

---

## Step 2: Identify target star and comparison stars

🕔 Estimated time: 4 minutes

In this multi-part step, you will use your stargazing skills to identify coordinates for your target star (HAT-P-32). You will:

- Watch the Video: "How to determine a star's coordinates" (2 minutes)
- Try it yourself: Enter the coordinates for the target and 

comparison stars (2 minutes)

In [ ]:
#@title <font size=3><img src="https://exoplanets.nasa.gov/system/exotic/leftdownarrow_tall.png" height=18 hspace=8>2a. Watch the video: "How to enter a star's coordinates"</font>

importCustomStyles()

display(HTML('<h3>VIDEO: How to determine a star\'s coordinates</h3>'));
display(HTML('<p>Be sure your audio is on.</p>'))

display(HTML('<video width="840" controls="true" autoplay src="https://exoplanets.nasa.gov/system/exotic/exotic-identify-stars.mov"/>'))

In [ ]:
#@title <font size=3><img src="https://exoplanets.nasa.gov/system/exotic/leftdownarrow_tall.png" height=18 hspace=8>2b. Identify the target and comparison stars in a telescope image</font>

importCustomStyles()

#########################################################

from exotic.api.colab import *

display(HTML('<p class="bookend">START: Compare telescope image and star chart</p>'))

display(HTML('<ul class="step4">'))

bokeh.io.output_notebook()
sample_data = False

p = "sample-data/HatP32Dec202017"
p = check_dir(os.path.join("/content/EXOTIC/exotic-in-action/", p))
output_dir = os.path.join(p, "EXOTIC_output/")      
                                         
inits = []    # array of paths to any inits files found in the directory

all_files_print = [f for f in sorted(os.listdir(p))]
all_files = [f for f in sorted(os.listdir(p)) if os.path.isfile(os.path.join(p, f))]
fits_count, first_image = 0, ""
for f in all_files:
  if re.search(r"\.f[itz]+s?\.?g?z?$", f, re.IGNORECASE):
    if first_image == "":
      first_image = os.path.join(p, f)
    fits_count += 1
  if re.search(r"\.json$", f, re.IGNORECASE):
    inits.append(os.path.join(p, f))

#print('Found ' + str(fits_count) + ' image files and ' + str(len(inits)) + ' initialization files in the directory')

if fits_count >= 2:
  obs = ""
  display(HTML('<h3>Data Entry 1 of 2: Enter coordinates for the target star</h3>'))
  display(HTML('<p>Tip: Use the zoom feature. Click the magnifying glass and click-and-drag to draw a rectangle that matches the starchart.</p>'))
  display(HTML('<ol class="step"><li class="step4">In the right image, find the <i>crosshairs</i> in the center - that represents your target star.</li><li class="step4">On the left image, <i>find this target star and roll over it with your mouse</i>, note the X and Y coordinates.</li><li class="step4">Put the X and Y coordinates in the box below in the format <code>[x,y]</code> and press return.</li></ol>')) 
  
  display(HTML('<hr />'))
  display(HTML('<div class="plots">'))
  display(HTML('<img align=right src="https://app.aavso.org/vsp/chart/X28194FDL.png" width=380>'))
  display_image(first_image)
  display(HTML('</div>'))
  display(HTML('<br clear="all"/>'))

  # verify the entry is good enough
  success = False
  while not success:
    targ_coords = input("Enter coordinates for target star - [424,286] - and press return:  ")  

    # check syntax and coords
    tc_syntax = re.search(r"\[\d+,\d+\]$", targ_coords)
    if tc_syntax:
      #display(HTML('<p class="output">Syntax OK: [x,y] e.g. ' + targ_coords + '</p>'))
      tc_coords = re.findall("\d+", targ_coords)
      if 422 <= int(tc_coords[0]) <= 426 and 284 <= int(tc_coords[1]) <= 288:
        #display(HTML('<p class="output">Coordinates OK: ' + targ_coords + ' are close to [424,286]</p>'))
        success = True
      else:
        display(HTML('<p class="output error">Try again, your coordinates are a bit off: ' + targ_coords + ' should be closer to [424,286]</p>'))
    else:
      display(HTML('<p class="output error">Try again, your syntax is not quite right: ' + targ_coords + ' needs to look like [424,286]</p>'))

  # if targ_coords != "[424,286]":
  #   display(HTML('<p class="output">You entered ' + targ_coords + '</p>'))
  # targ_coords = "[424,286]"


  showProgress(1)
  display(HTML('<p class="output">Target star coordinates saved to inits.json</p>'))
  
  display(HTML('<h3>Data Entry 2 of 2: Enter coordinates for at least two comparison stars.</h3>'))
  display(HTML('<ol class="step"><li class="step4">In the right image, find the stars <i>with numbers</i> that represent suggested comparison stars.</li><li class="step4">On the left image, <i>find and roll over each comparison star with your mouse</i> and note the coordinates.</li><li class="step4">Put the X and Y coordinates in the box below in the format <code>[[x1,y1][x2,y2]]</code> and press return.</li></ol>'))
  
  # verify the entry is good enough
  success = False
  while not success:
    comp_coords = input("Enter coordinates for the comparison stars - [[326,365],[416,343]] - and press return:  ")  

    # check syntax
    tc_syntax = re.search(r"\[(\[\d+,\d+\],?)+\]$", comp_coords)
    if tc_syntax:
      #display(HTML('<p class="output">Syntax OK:  [[x1,y1],[x2,y2]] e.g. ' + comp_coords + '</p>'))
      success = True
    else:
      display(HTML('<p class="output error">Try again, your syntax is not quite right: ' + comp_coords + ' needs to look more like [[326,365],[416,343]] (make sure you have all the brackets!)</p>'))

  # if comp_coords != "[[326,365],[416,343],[491,303]]":
  #   display(HTML('<p class="step">You entered ' + comp_coords + '<br /><br /></p>'))
  # comp_coords = "[[326,365],[416,343],[491,303]]"

  showProgress(1)
  display(HTML('<p class="output">Comparison star coordinates saved to inits.json</p>'))
  display(HTML('<p class="output">Images and inits.json set up. Ready for EXOTIC data reduction/analysis</p>'))
  aavso_obs_code = ""

  if not os.path.isdir(output_dir):
    os.mkdir(output_dir)

  #inits = [make_inits_file(planetary_params, p, output_dir, first_image, targ_coords, comp_coords, obs, aavso_obs_code, sample_data)]
  inits = [output_dir+"inits.json"]

  if not os.path.isdir(output_dir):    # Make the output directory if it does not exist already.
    os.mkdir(output_dir)
  output_dir_for_shell = output_dir.replace(" ", "\ ")

#print("Path to the inits file(s) that will be used:")

#for inits_file in inits:
#  print(inits_file)

num_inits = len(inits)


display(HTML('<p class="bookend">DONE: Compare telescope image and star chart. <b>You may move on to the next step.</b></p>'))




# **Step 5:  Run EXOTIC**

---
> Estimated time: 5 minutes
---

EXOTIC will analyze the brightness of the target star in each telescop image to identify if there was a transit. At the end of this step, you will see your lightcurve as an image, and the lightcurve data will be downloaded to your hard drive for submission to AAVSO.

In [ ]:
#@title <font face="Helvetica" class="button" color='#702020'>&lt;- Click to run EXOTIC to analyze telescope images (unstyled, simulated)</font>

display(HTML('<p class="bookend">START: Analyzing telescope images</p>'))
setupDisplay()
display(HTML('<ul class="step_container"></ul>'))
log_10 = """
Path to the inits file(s) that will be used:
/content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/inits.json
!exotic -red "/content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/inits.json" -ov
    Downloading __databases__.pickle_new...
    Done!
Checking exotethys database...
Checking ephemerides database...
Checking photometry database...
Checking catalogues database...

*************************************************************
Welcome to the EXOplanet Transit Interpretation Code (EXOTIC)
Version 1.10.2
*************************************************************


**************************
Complete Reduction Routine
**************************

**************************
Starting Reduction Process
**************************


Getting the plate solution for your imaging file to translate pixel coordinates on the sky. 
Please wait....

"""

log_20 = """
WCS file creation successful.
Thinking ... DONE!

Here is the path to your plate solution: /content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/wcs.fits

Checking for variability in Comparison Star #1:
	Pixel X: 493 Pixel Y: 304
 Warning: Your comparison star cannot be resolved in the SIMBAD star database; EXOTIC cannot check if it is variable or not. 
EXOTIC will still include this star in the reduction. 
Please proceed with caution as we cannot check for stellar variability.


Checking for variability in Comparison Star #2:
	Pixel X: 415 Pixel Y: 343
 Warning: Your comparison star cannot be resolved in the SIMBAD star database; EXOTIC cannot check if it is variable or not. 
EXOTIC will still include this star in the reduction. 
Please proceed with caution as we cannot check for stellar variability.


Computing best comparison star, aperture, and sky annulus. Please wait.
"""

log_30 = """

*********************************************
Best Comparison Star: None
Minimum Residual Scatter: 0.672%
Optimal Aperture: 5.02
Optimal Annulus: 7.54
*********************************************

No Barycentric Julian Dates (BJDs) in Image Headers for standardizing time format. Converting all JDs to BJD_TDBs.
Please be patient- this step can take a few minutes.
"""

log_40 = """
Thinking ... DONE!


Output File Saved


****************************************
Fitting a Light Curve Model to Your Data
****************************************

[ultranest] Sampling 400 live points from prior ...


Mono-modal Volume: ~exp(-3.81) * Expected Volume: exp(0.00) Quality: ok

rprs:   +0.0000|**********|  +0.1861
tmid: +2458107.706|**********|+2458107.716
ars :      +4.8|**********|     +5.9
a2  :      -3.0|**********|     +3.0

Z=-1659103.7(0.00%) | Like=-1655784.03..-111.41 [-8987932.8990..-176103.1770] | it/evals=80/494 eff=85.1064% N=400 

Mono-modal Volume: ~exp(-3.96) * Expected Volume: exp(-0.23) Quality: ok

rprs:   +0.0000|**********|  +0.1861
tmid: +2458107.706|**********|+2458107.716
ars :      +4.8|**********|     +5.9
a2  :      -3.0|********  |     +3.0

Z=-338377.3(0.00%) | Like=-334422.26..-111.41 [-8987932.8990..-176103.1770] | it/evals=160/609 eff=76.5550% N=400 

Mono-modal Volume: ~exp(-4.18) * Expected Volume: exp(-0.45) Quality: ok

rprs:   +0.0000|**********|  +0.1861
tmid: +2458107.706|**********|+2458107.716
ars :      +4.8|**********|     +5.9
a2  :      -3.0|*******   |     +3.0

Z=-202573.2(0.00%) | Like=-201882.30..-111.41 [-8987932.8990..-176103.1770] | it/evals=240/730 eff=72.7273% N=400 

"""

log_50 = """
Mono-modal Volume: ~exp(-20.47)   Expected Volume: exp(-17.10) Quality: ok

rprs:    +0.000|        * |   +0.186
tmid: +2458107.706|       ** |+2458107.716
ars :      +4.8|    **    |     +5.9
a2  :    -1.000|    *     |   +1.000

[ultranest] Explored until L=-7e+01  
[ultranest] Likelihood function evaluations: 15883
[ultranest]   logZ = -82.92 +- 0.141
[ultranest] Effective samples strategy satisfied (ESS = 2176.7, need >400)
[ultranest] Posterior uncertainty strategy is satisfied (KL: 0.45+-0.05 nat, need <0.50 nat)
[ultranest] Evidency uncertainty strategy is satisfied (dlogz=0.14, need <0.5)
[ultranest]   logZ error budget: single: 0.16 bs:0.14 tail:0.01 total:0.14 required:<0.50
[ultranest] done iterating.

*********************************************************
FINAL PLANETARY PARAMETERS

          Mid-Transit Time [BJD_TDB]: 2458107.7136 +/- 0.001
  Radius Ratio (Planet/Star) [Rp/Rs]: 0.1587 +/- 0.0037
           Transit depth [(Rp/Rs)^2]: 2.52 +/- 0.12 [%]
 Semi Major Axis/ Star Radius [a/Rs]: 5.36 +/- 0.11
               Airmass coefficient 1: 1.1645 +/- 0.0078
               Airmass coefficient 2: -0.1222 +/- 0.0028
                    Residual scatter: 0.67 %
                 Best Comparison Star: None
                    Optimal Aperture: 5.02
                     Optimal Annulus: 7.54
              Transit Duration [day]: 0.1311 +/- 0.0028
*********************************************************
Output Files Saved

************************
End of Reduction Process
************************
"""

log_60 = """
************************
EXOTIC has successfully run!!!
It is now safe to close this window.
************************
lightcurve: /content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/FinalLightCurve_HAT-P-32 b_2017-12-19.png
"""

import time
print(log_10)
showProgress(3)
print(log_20)
showProgress(2)
print(log_30)
showProgress(5)
print(log_40)
showProgress(1)
print(log_50)
showProgress(4)
print(log_60)

display(HTML('<p>The data for the lightcurve you see here is at /content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/AAVSO_HAT-P-32 b_2017-12-19.txt'))
display(HTML('<p>Downloading now...</p>'))

showProgress(1)

if os.path.isfile('/content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/AAVSO_HAT-P-32 b_2017-12-19.txt'):
  files.download('/content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/AAVSO_HAT-P-32 b_2017-12-19.txt')
else: 
  display(HTML('<p>Couldn\'t find output file. Bergen will work with Rob to ensure it is in there.</p>'))


display(HTML('<p class="bookend">DONE: Analyzing telescope images. <b>You may move on to the next step.</b></p>'))



---

---

# **DONE: Next Steps** 

You have successfully generated a lightcurve showing the transit of Hat-p-32b around Hat-p-32 and downloaded the data to your hard drive as "AAVSO_HAT-P-32 b_2017-12-19.txt", in a format suitable for submission to AAVSO. 

This exoplanet was actually discovered in 2011, and you can [explore it in 3D here](https://exoplanets.nasa.gov/exoplanet-catalog/1434/hat-p-32-b/).

Ready for the next step?  [Use EXOTIC to upload your own data!](https://exoplanets-5.client.mooreboeck.com/exoplanet-watch/exotic/)

---

---

# **APPENDIX: BONUS SEGMENTS**

In [ ]:
#@title <font face="Helvetica" class="button" color='#702020'>&lt;- Click to run EXOTIC to analyze telescope images (developer's note, unstyled, real)</font>

# If the user presses enter to run the sample data, download sample data if needed and
# put it into a sample-data directory at the top level of the user's Gdrive.  Count
# the .fits files (images) and .json files (inits files) in the directory entered 
# by the user (or in the sample-data directory if the user pressed enter).  If 
# there are at least 20 .fits files, assume this is a directory of images and display
# the first one in the series.  If there is exactly one inits file in the directory, 
# show the specified target and comp coords so that the user can check these against
# the displayed image.  Otherwise, prompt for target / comp coords and make an inits 
# file based  on those (save this new inits file in the folder with the output files 
# so that the student can consult it later).  Finally, run EXOTIC with the newly-made 
# or pre-existing inits file, plus any other inits files in the directory.

#########################################################

# p is the name of the folder entered by the user.  Decide what to do based on what
# is found in the folder.
display(HTML('<p class="bookend">START: Analyzing telescope images</p>'))
#display(HTML("<p class='warning'>NOTE: At this point in EXOTIC, you would have the opportunity choose where to temporarily save the sample data. For this exercise, we're downloading to /content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017"))
setupDisplay()
display(HTML('<ul class="step_container"></ul>'))
#bokeh.io.output_notebook()
sample_data = False

print("Path to the inits file(s) that will be used:")

for inits_file in inits:
  print(inits_file)

num_inits = len(inits)

commands = []
for inits_file in inits:
  with open(inits_file) as i_file:
    inits_data = i_file.read()
    d = json.loads(inits_data)
    date_obs = d["user_info"]["Observation date"]
    planet = d["planetary_parameters"]["Planet Name"]
    output_dir = d["user_info"]["Directory to Save Plots"]
    if not os.path.isdir(output_dir):
      os.mkdir(output_dir)
    inits_file_for_shell = inits_file.replace(" ", "\\ ")
    run_exotic = str(f"exotic -red {inits_file_for_shell} -ov")
    debug_exotic_run = str(f"!exotic -red \"{inits_file}\" -ov")

    commands.append({"inits_file_for_shell": inits_file_for_shell, "output_dir": output_dir, 
                      "planet": planet, "date_obs": date_obs, 
                      "run_exotic": run_exotic, "debug_exotic_run": debug_exotic_run
                      })
    print(f"{debug_exotic_run}")
    !eval "$run_exotic"

    # Only show lightcurve for beginner - bm
    lightcurve = os.path.join(output_dir,"FinalLightCurve_"+planet+"_"+date_obs+".png")
    fov = os.path.join(output_dir,"temp/FOV_"+planet+"_"+date_obs+"_LinearStretch.png")
    triangle = os.path.join(output_dir,"temp/Triangle_"+planet+"_"+date_obs+".png")
    print(f"lightcurve: {lightcurve}\nfov: {fov}\ntriangle: {triangle}\n")

    if not (os.path.isfile(lightcurve) and os.path.isfile(fov) and os.path.isfile(triangle)):
      print(f"Something went wrong with {planet} {date_obs}.\nCopy the command below into a new cell and run to find the error:\n{debug_exotic_run}\n")
      continue

    imageA = widgets.Image(value=open(lightcurve, 'rb').read())
    imageB = widgets.Image(value=open(fov, 'rb').read())
    hbox = HBox([imageB, imageA])
    # removing for "see it in action" - bm
    display(imageA)
    #display(hbox)
    #display(Image(filename=triangle))

display(HTML('<p>The data for the lightcurve you see here is downloading now in a format suitable to submit to AAVSO.'))

# /content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/AAVSO_HAT-P-32 b_2017-12-19.txt

showProgress(2)

if os.path.isfile('/content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/AAVSO_HAT-P-32 b_2017-12-19.txt'):
  files.download('/content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/AAVSO_HAT-P-32 b_2017-12-19.txt')
else: 
  display(HTML('<p>Couldn\'t find output file. Bergen will work with Rob to ensure it is in there.</p>'))


display(HTML('<p class="bookend">DONE: Analyzing telescope images. <b>You may move on to the next step.</b></p>'))

    

In [ ]:
#@title <font face="Helvetica" class="button" color='#702020'>&lt;- Click to run EXOTIC to analyze sample data (developer's note: styled, simulated)</font>

display(HTML('<p class="bookend">START: Analyzing telescope images</p>'))
setupDisplay()
display(HTML('<ul class="step_container"></ul>'))

log_10 = """
Path to the inits file(s) that will be used:
/content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/inits.json
!exotic -red "/content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/inits.json" -ov
    Downloading __databases__.pickle_new...
    Done!
Checking exotethys database...
Checking ephemerides database...
Checking photometry database...
Checking catalogues database...

*************************************************************
Welcome to the EXOplanet Transit Interpretation Code (EXOTIC)
Version 1.10.2
*************************************************************


**************************
Complete Reduction Routine
**************************

**************************
Starting Reduction Process
**************************


Getting the plate solution for your imaging file to translate pixel coordinates on the sky. 
Please wait....

"""

log_20 = """
WCS file creation successful.
Thinking ... DONE!

Here is the path to your plate solution: /content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/wcs.fits

Checking for variability in Comparison Star #1:
	Pixel X: 493 Pixel Y: 304
 Warning: Your comparison star cannot be resolved in the SIMBAD star database; EXOTIC cannot check if it is variable or not. 
EXOTIC will still include this star in the reduction. 
Please proceed with caution as we cannot check for stellar variability.


Checking for variability in Comparison Star #2:
	Pixel X: 415 Pixel Y: 343
 Warning: Your comparison star cannot be resolved in the SIMBAD star database; EXOTIC cannot check if it is variable or not. 
EXOTIC will still include this star in the reduction. 
Please proceed with caution as we cannot check for stellar variability.


Computing best comparison star, aperture, and sky annulus. Please wait.
"""

log_30 = """

*********************************************
Best Comparison Star: None
Minimum Residual Scatter: 0.672%
Optimal Aperture: 5.02
Optimal Annulus: 7.54
*********************************************

No Barycentric Julian Dates (BJDs) in Image Headers for standardizing time format. Converting all JDs to BJD_TDBs.
Please be patient- this step can take a few minutes.
"""

log_40 = """
Thinking ... DONE!


Output File Saved


****************************************
Fitting a Light Curve Model to Your Data
****************************************

[ultranest] Sampling 400 live points from prior ...


Mono-modal Volume: ~exp(-3.81) * Expected Volume: exp(0.00) Quality: ok

rprs:   +0.0000|**********|  +0.1861
tmid: +2458107.706|**********|+2458107.716
ars :      +4.8|**********|     +5.9
a2  :      -3.0|**********|     +3.0

Z=-1659103.7(0.00%) | Like=-1655784.03..-111.41 [-8987932.8990..-176103.1770] | it/evals=80/494 eff=85.1064% N=400 

Mono-modal Volume: ~exp(-3.96) * Expected Volume: exp(-0.23) Quality: ok

rprs:   +0.0000|**********|  +0.1861
tmid: +2458107.706|**********|+2458107.716
ars :      +4.8|**********|     +5.9
a2  :      -3.0|********  |     +3.0

Z=-338377.3(0.00%) | Like=-334422.26..-111.41 [-8987932.8990..-176103.1770] | it/evals=160/609 eff=76.5550% N=400 

Mono-modal Volume: ~exp(-4.18) * Expected Volume: exp(-0.45) Quality: ok

rprs:   +0.0000|**********|  +0.1861
tmid: +2458107.706|**********|+2458107.716
ars :      +4.8|**********|     +5.9
a2  :      -3.0|*******   |     +3.0

Z=-202573.2(0.00%) | Like=-201882.30..-111.41 [-8987932.8990..-176103.1770] | it/evals=240/730 eff=72.7273% N=400 

"""

log_50 = """
Mono-modal Volume: ~exp(-20.47)   Expected Volume: exp(-17.10) Quality: ok

rprs:    +0.000|        * |   +0.186
tmid: +2458107.706|       ** |+2458107.716
ars :      +4.8|    **    |     +5.9
a2  :    -1.000|    *     |   +1.000

[ultranest] Explored until L=-7e+01  
[ultranest] Likelihood function evaluations: 15883
[ultranest]   logZ = -82.92 +- 0.141
[ultranest] Effective samples strategy satisfied (ESS = 2176.7, need >400)
[ultranest] Posterior uncertainty strategy is satisfied (KL: 0.45+-0.05 nat, need <0.50 nat)
[ultranest] Evidency uncertainty strategy is satisfied (dlogz=0.14, need <0.5)
[ultranest]   logZ error budget: single: 0.16 bs:0.14 tail:0.01 total:0.14 required:<0.50
[ultranest] done iterating.

*********************************************************
FINAL PLANETARY PARAMETERS

          Mid-Transit Time [BJD_TDB]: 2458107.7136 +/- 0.001
  Radius Ratio (Planet/Star) [Rp/Rs]: 0.1587 +/- 0.0037
           Transit depth [(Rp/Rs)^2]: 2.52 +/- 0.12 [%]
 Semi Major Axis/ Star Radius [a/Rs]: 5.36 +/- 0.11
               Airmass coefficient 1: 1.1645 +/- 0.0078
               Airmass coefficient 2: -0.1222 +/- 0.0028
                    Residual scatter: 0.67 %
                 Best Comparison Star: None
                    Optimal Aperture: 5.02
                     Optimal Annulus: 7.54
              Transit Duration [day]: 0.1311 +/- 0.0028
*********************************************************
Output Files Saved

************************
End of Reduction Process
************************
"""

log_60 = """
************************
EXOTIC has successfully run!!!
It is now safe to close this window.
************************
lightcurve: /content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/FinalLightCurve_HAT-P-32 b_2017-12-19.png
"""


displayStep('Using init file at /content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/inits.json')

displayStep('Checking exotethys database...')
displayStep('Checking ephemerides database...')
displayStep('Checking photometry database...')
displayStep('Checking catalogues database...')

showProgress(2)
display(HTML('<pre class="log">' + log_10 + ' </pre>'))
showProgress(1)
displayStep('Starting Reduction Process')

displayStep('Getting the plate solution for your imaging file to translate pixel coordinates on the sky.')
showProgress(2)

display(HTML('<pre class="log">' + log_20 + ' </pre>'))

displayStep('WCS file creation successful.')
displayStep('Checking for variability in Comparison Star #1:')
showProgress(1)
displayStep('Checking for variability in Comparison Star #2:')
showProgress(1)

display(HTML('<pre class="log">' + log_30 + ' </pre>'))

displayStep('Computing best comparison star, aperture, and sky annulus. Please wait.')
showProgress(2)

display(HTML('<pre class="log">' + log_40 + ' </pre>'))

displayStep('Converting all JDs to BJD_TDBs.')
showProgress(1)


displayStep('Output File Saved')

display(HTML('<pre class="log">' + log_50 + ' </pre>'))

displayStep('Fitting a Light Curve Model to Your Data')
showProgress(3)

displayStep('Lightcurve Output Files Saved')

display(HTML('<pre class="log">' + log_60 + ' </pre>'))

# print(log_10)
# time.sleep(3)
# print(log_20)
# time.sleep(5)
# print(log_30)
# time.sleep(9)
# print(log_40)
# time.sleep(1)
# print(log_50)
# time.sleep(7)
# print(log_60)

display(HTML('<br /><img src="https://exoplanets.nasa.gov/system/exotic/sample_lightcurve.png" width=500><br />'))


display(HTML('<p>The data for the lightcurve you see here is downloading now in a format suitable to submit to AAVSO.'))

# /content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/AAVSO_HAT-P-32 b_2017-12-19.txt

showProgress(2)

if os.path.isfile('/content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/AAVSO_HAT-P-32 b_2017-12-19.txt'):
  files.download('/content/EXOTIC/exotic-in-action/sample-data/HatP32Dec202017/EXOTIC_output/AAVSO_HAT-P-32 b_2017-12-19.txt')
else: 
  display(HTML('<p>Couldn\'t find output file. Bergen will work with Rob to ensure it is in there.</p>'))

display(HTML('<p class="bookend">DONE: Analyzing telescope images. <b>You may move on to the next step.</b></p>'))



🏴 🎌 🔴 🟥 🔻 🔺 ⛑ 🚨 🛑 🎯 🚩